In [72]:
import unidecode
from nltk.translate.bleu_score import sentence_bleu
import random as rd

play_file_infixes = ['othello', 'antony-and-cleopatra', 'asyoulikeit', 
                     'errors', 'hamlet', 'henryv', 'juliuscaesar', 'lear', 'macbeth', 
                     'merchant', 'msnd', 'muchado', 'richardiii', 'romeojuliet', 
                     'shrew', 'tempest', 'twelfthnight']

agg_original_tuples = []
agg_modern_tuples = []
path = "data/shakespeare/data/align/plays/merged/"

for infix in play_file_infixes:
    modern_tuples = unidecode.unidecode(open(path + infix + "_modern.snt.aligned").read()).split("\n")
    original_tuples = unidecode.unidecode(open(path + infix + "_original.snt.aligned").read()).split("\n")
    agg_original_tuples.extend(original_tuples)
    agg_modern_tuples.extend(modern_tuples)
# aligned_othello_style1 = "data/shakespeare/data/align/plays/merged/othello_modern.snt.aligned"
# aligned_othello_style2 = "data/shakespeare/data/align/plays/merged/othello_original.snt.aligned"

# aligned_ant_cleo_style1 = "data/shakespeare/data/align/plays/merged/antony-and-cleopatra_modern.snt.aligned"
# aligned_ant_cleo_style2 = "data/shakespeare/data/align/plays/merged/antony-and-cleopatra_original.snt.aligned"

# aligned_asyoulikeit_style1 = "data/shakespeare/data/align/plays/merged/asyoulikeit_modern.snt.aligned"
# aligned_asyoulikeit_style2 = "data/shakespeare/data/align/plays/merged/asyoulikeit_original.snt.aligned"

# aligned_errors_style1 = "data/shakespeare/data/align/plays/merged/errors_modern.snt.aligned"
# aligned_errors_style2 = "data/shakespeare/data/align/plays/merged/errors_original.snt.aligned"

# aligned_hamlet_style1 = "data/shakespeare/data/align/plays/merged/hamlet_modern.snt.aligned"
# aligned_hamlet_style2 = "data/shakespeare/data/align/plays/merged/hamlet_original.snt.aligned"

# aligned_henryv_style1 = "data/shakespeare/data/align/plays/merged/henryv_modern.snt.aligned"
# aligned_henryv_style2 = "data/shakespeare/data/align/plays/merged/henryv_original.snt.aligned"

# aligned_juliuscaesar_style1 = "data/shakespeare/data/align/plays/merged/juliuscaesar_modern.snt.aligned"
# aligned_juliuscaesar_style2 = "data/shakespeare/data/align/plays/merged/juliuscaesar_original.snt.aligned"

# style1_tuples = unidecode.unidecode(open(aligned_style1).read())
# style2_tuples = unidecode.unidecode(open(aligned_style2).read())

# style1_tuples = style1_tuples.split("\n")
# style2_tuples = style2_tuples.split("\n")

In [73]:
print("{}\n{}\n".format(agg_original_tuples[0], agg_modern_tuples[0]))
print("{}\n{}\n".format(agg_original_tuples[5000], agg_modern_tuples[5000]))
print("{}\n{}\n".format(agg_original_tuples[9000], agg_modern_tuples[9000]))
print("{}\n{}\n".format(agg_original_tuples[18000], agg_modern_tuples[18000]))


What's the matter, lieutenant?
What's the matter, lieutenant?

Tell me this, I pray: Where have you left the money that I gave you?
Answer me this, please: where's the money I gave you?

Lucius, who's that knocking?
Lucius, who's that knocking?

Gentlemen, forward to the bridal dinner.
Gentlemen, on to the bridal dinner.



In [74]:
# in-class = original = '1'
# raw -> (raw text, label)
import numpy as np

style1_np = np.array(agg_original_tuples)
style1_labels = np.array(['1' for x in range(len(agg_original_tuples))])
X = np.vstack((style1_np, style1_labels))

style2_np = np.array(agg_modern_tuples)
style2_labels = np.array(['0' for x in range(len(agg_modern_tuples))])
X2 = np.vstack((style2_np, style2_labels))

X = np.hstack((X,X2))
raw = np.transpose(X)
raw.shape

idx_col = [str(x) for x in range(len(raw))]
idx_col = np.array(idx_col).reshape(-1,1)
print(idx_col.shape)
print(raw.shape)
raw = np.hstack((idx_col, raw))
raw.shape

(42192, 1)
(42192, 2)


(42192, 3)

In [21]:
raw[2000]

array(['2000', 'We looked not for Mark Antony here.', '1'], dtype='<U549')

In [75]:
from sklearn.feature_extraction import DictVectorizer

vectorizer = DictVectorizer()
features = []


# feature engineering
    # construct index column
idx_col = raw[:,0]

# import spacy
# nlp = spacy.load('en_core_web_sm')
# stri = 'Go bid the priests do present sacrifice And bring me their opinions of success'
# doc = nlp(stri)

# for token in doc:
#     print("{}\t{}    {} {}".format(token.text, token.pos_, token.tag_, token.dep_))

# adj_count
# adverb_count
# verb_count
# noun_count
# adv_verb_ratio
# adj_noun ratio

# def pos_
    
sent_len_feature = np.array([-1 for x in range(len(raw))])
raw_text_col = 1
LABEL_COL = -1

for row_idx,_ in enumerate(raw):
    sent_len_feature[row_idx] = len(raw[row_idx][raw_text_col])
    features.append({'sent_len': len(raw[row_idx][raw_text_col])})
    
#     if sent_len_feature[row_idx] < 0:
#         print("sent len less than zero!")

X = vectorizer.fit_transform(features)

X = np.vstack((idx_col,sent_len_feature, raw[:,LABEL_COL].astype(float)))
X = np.transpose(X)
X = X.astype('f', copy=True)
# print(X[0:5])
# print(X[3700:3705])

In [76]:
X[0]

array([ 0., 30.,  1.], dtype=float32)

In [77]:
from sklearn.linear_model import LogisticRegression
import time
import math
# training
# test breakout: 3500, 210
train_test_split_delim = math.floor(len(X) * .8)
print("total rows: {} traintest split delim: {}".format(len(X), train_test_split_delim))
# numpy note: addressing [:delim] is exclusive of delim, but
#                         [delim:] is inclusive of delim

X = np.random.permutation(X)

X_train = X[0:train_test_split_delim,1]
X_train = X_train.reshape(-1, 1) #TODO: REMOVE
Y_train = X[0:train_test_split_delim,LABEL_COL]

X_test = X[train_test_split_delim:,1]
X_test = X_test.reshape(-1, 1)
Y_test = X[train_test_split_delim:,LABEL_COL]

logreg = LogisticRegression(random_state=123)
logreg.fit(X_train, Y_train)

np.set_printoptions(suppress=True)
X[0:10]

total rows: 42192 traintest split delim: 33753


array([[27277.,    24.,     0.],
       [22474.,    32.,     0.],
       [15408.,   124.,     1.],
       [14384.,   241.,     1.],
       [19001.,    81.,     1.],
       [  197.,    82.,     1.],
       [ 1336.,    19.,     1.],
       [14788.,    44.,     1.],
       [ 3296.,    27.,     1.],
       [32396.,    18.,     0.]], dtype=float32)

In [79]:
# evaluation
preds = logreg.predict(X_test)
test_indices = X[train_test_split_delim:,0]
if len(preds) != len(test_indices):
    raise ValueError("unmatched arrays")
    
results = []
for (idx, pred) in zip(test_indices,preds):
    gold_label = int(raw[int(idx)][-1])
    pred_label = int(pred)
    #print("int'd idx: {} gold label: {} pred label: {}".format(int(idx), gold_label, pred_label))
    
    if gold_label == pred_label:
          results.append(1)
    else:
        results.append(0)
    
results[:5]


[1, 0, 1, 1, 1]

In [80]:
test_len = len(preds)
accuracy = sum(preds)/float(test_len)
print("accuracy: {}".format(accuracy))

#othello accuracy: 0.7428571428571429. but maybe not permuted
#all: accuracy: 0.328237942884228


accuracy: 0.3967294703163882


In [81]:
#x = np.zeros(3, dtype=[('row_id','int32'),('text', 'S'), ('label', 'int32')])

In [82]:
raw[9000:9002]

array([['9000', "Lucius, who's that knocking?", '1'],
       ['9001', 'He is a sick man that would speak with you.', '1']],
      dtype='<U549')

In [47]:
# import spacy
# nlp = spacy.load('en_core_web_sm')
# stri = 'Go bid the priests do present sacrifice And bring me their opinions of success'
# doc = nlp(stri)

# for token in doc:
#     print("{}\t{}    {} {}".format(token.text, token.pos_, token.tag_, token.dep_))

Go	VERB    VB aux
bid	VERB    VB ROOT
the	DET    DT det
priests	NOUN    NNS nsubj
do	VERB    VBP ccomp
present	ADJ    JJ amod
sacrifice	NOUN    NN dobj
And	CCONJ    CC cc
bring	VERB    VB conj
me	PRON    PRP dative
their	ADJ    PRP$ poss
opinions	NOUN    NNS dobj
of	ADP    IN prep
success	NOUN    NN pobj
